In [1]:
#import packages to build the game
import pygame
import time
import os
import random

#initialize pygame
pygame.init()

#set up the window to display the game
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 550
WINDOW = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))

#set up the font
FONT = pygame.font.SysFont('comicsansms', 20)

#load the required images
BIRD_IMGS = [pygame.image.load('flappy bird.png'),
             pygame.image.load('flappy bird wings up.png'),
             pygame.image.load('flappy bird wings down.png')]
BOTTOM_PIPE_IMG = pygame.image.load('bottom pipe.png')
TOP_PIPE_IMG = pygame.transform.flip(BOTTOM_PIPE_IMG, False, True) #flip the image of the bottom pipe to get the image for the pipe on the top
FLOOR_IMG = pygame.image.load('floor2.png')
BG_IMG = pygame.transform.scale(pygame.image.load('background2.png'), (WINDOW_WIDTH, WINDOW_HEIGHT))

#set the game options
FPS = 30 #run the game at rate FPS, the speed at which images are shown
max_score = 100 #the maximum score of the game before we break the loop

#floor options
floor_velocity = 5 #the horizontal moving velocity of the floor, this should equal to pipe_velocity
floor_starting_y_position = 500 #the starting y position of the floor

#pipe options
pipe_max_num = 100 #the maximum number of pipes in this game
pipe_vertical_gap = 150 #the gap between the top pipe and the bottom pipe, the smaller the number, the harder the game
pipe_horizontal_gap = 200 #the gap between two sets of pipes
pipe_velocity = 5 #the horizontal moving velocity of the pipes, this should equal to floor_velocity
top_pipe_min_height = 100 #the minimum height of the top pipe (carefully set this number)
top_pipe_max_height = 300 #the maximum height of the top pipe (carefully set this number)
pipe_starting_x_position = 500 #the starting x position of the first pipe

#bird options
bird_min_upward_rotation_degree = 35 #the minimum upward rotation degree
bird_downward_rotation_velocity = 10 #the rotation velocity when going downward
bird_flap_time = 2 #the animation time of showing one image
bird_jump_velocity = -8 #the vertical jump up velocity
bird_acceleration = 3 #the gravity for the bird in the game
bird_max_displacement = 12 #the maximum displacement per frame
bird_starting_x_position = 150 #the starting x position of the bird
bird_starting_y_position = 250 #the starting y position of the bird

#NEAT options
generation = 0 #generation 0 is our first generation
MAX_GEN = 50 #the maximum number of generation to run
pass_reward = 5 #the amount of fitness increase after pass the pipe
stay_reward = 0.1 #the amount of fitness increase for staying 1 frame longer
prob_threshold_to_jump = 0.8 #the probability threshold to activate the bird to jump
failed_punishment = 10 #the amount of fitness decrease after collision

#build the class Floor
class Floor:
    #Floor's attributes
    VELOCITY = floor_velocity #the moving velocity of the floor
    IMG_WIDTH = FLOOR_IMG.get_width() #the width of the floor

    #initialize the Object
    def __init__(self, y_position):
        #since our floor image is not wide enough to fill the screen, we need 3 floor images to set up the moving floor
        #these 3 images have different starting position but have the same y position
        self.floor1 = FLOOR_IMG
        self.floor2 = FLOOR_IMG
        self.floor3 = FLOOR_IMG
        self.x1 = 0 #the starting x position of the first floor image
        self.x2 = self.IMG_WIDTH #the starting x position of the second floor image
        self.x3 = self.IMG_WIDTH * 2 #the starting x position of the third floor image
        self.y = y_position #the y position of the floor image
        
    #define a function to move the floor
    def move(self):
        self.x1 -= self.VELOCITY #move to the left with the velocity of VELOCITY
        self.x2 -= self.VELOCITY #move to the left with the velocity of VELOCITY
        self.x3 -= self.VELOCITY #move to the left with the velocity of VELOCITY
        
        if self.x1 + self.IMG_WIDTH < 0: #if the first floor image moves out of the window 
            self.x1 = self.x3 + self.IMG_WIDTH #then move the first floor image to to the right of the third floor image
        if self.x2 + self.IMG_WIDTH < 0: #if the second floor image moves out of the window 
            self.x2 = self.x1 + self.IMG_WIDTH #then move the second floor image to to the right of the first floor image
        if self.x3 + self.IMG_WIDTH < 0: #if the third floor image moves out of the window 
            self.x3 = self.x2 + self.IMG_WIDTH #then move the third floor image to to the right of the second floor image
            
    #define a function to draw the floor
    def draw_surface(self, window):
        window.blit(self.floor1, (self.x1, self.y)) #draw the first floor image
        window.blit(self.floor2, (self.x2, self.y)) #draw the second floor image
        window.blit(self.floor3, (self.x3, self.y)) #draw the third floor image

#build the class Pipe
class Pipe:
    #Pipe's attributes
    VERTICAL_GAP = pipe_vertical_gap #the gap between the top and bottom pipes
    VELOCITY = pipe_velocity #the moving velocity of the pipes
    IMG_WIDTH = TOP_PIPE_IMG.get_width() #the width of the pipe
    IMG_LENGTH = TOP_PIPE_IMG.get_height() #the length of the pipe

    #initialize the Object
    def __init__(self, x_position):                
        self.top_pipe = TOP_PIPE_IMG #get the image for the pipe on the top
        self.bottom_pipe = BOTTOM_PIPE_IMG #get the image for the pipe on the bottom
        self.x = x_position #starting x position of the first set of pipes
        self.top_pipe_height = 0 #the height of the top pipe, initialized to be 0
        self.top_pipe_topleft = 0 #the topleft position of the top pipe, initialized to be 0
        self.bottom_pipe_topleft = 0 #the topleft position of the bottom pipe, initialized to be 0
        self.set_height() #set the height of the pipes randomly as well as the starting topleft position for top and bottom pipes
        
    #define a function to move the pipe
    def move(self):
        self.x -= self.VELOCITY
    
    def set_height(self):
        self.top_pipe_height = random.randrange(top_pipe_min_height, top_pipe_max_height) #the range is between top_pipe_min_height and top_pipe_max_height
        self.top_pipe_topleft = self.top_pipe_height - self.IMG_LENGTH #the topleft position of the top pipe should be the random height - the length of the pipe
        self.bottom_pipe_topleft = self.top_pipe_height + self.VERTICAL_GAP #the topleft position of the bottom pipe should be the random height + the GAP

    #define a function to draw the pipes
    def draw_surface(self, window):
        window.blit(self.top_pipe, (self.x, self.top_pipe_topleft)) #draw the pipe on the top
        window.blit(self.bottom_pipe, (self.x, self.bottom_pipe_topleft)) #draw the pipe on the bottom

#build the class Bird
class Bird:
    #Bird's attributes
    IMGS = BIRD_IMGS
    MIN_UPWARD_ROTATION_DEGREE = bird_min_upward_rotation_degree
    DOWNWARD_ROTATION_VELOCITY = bird_downward_rotation_velocity
    FLAP_TIME = bird_flap_time
    
    #initialize the Object
    def __init__(self, x_position, y_position):
        self.bird_img = self.IMGS[0] #use the first image as the initial image
        self.x = x_position #starting x position
        self.y = y_position #starting y position
        self.fly_degree = 0 #starting flying degree, initialized to be 0
        self.time = 0 #starting time set to calculate displacement, initialized to be 0
        self.velocity = 0 #starting velocity, initialized to be 0
        self.fly_height = self.y #starting height of the bird, initialized to be y
        self.animation_time_count = 0 #used to change bird images, initialized to be 0
        
    #defien a function to move the bird
    def move(self):
        self.time += 1 #count the time
        
        #for a body with a nonzero speed v and a constant acceleration a
        #the displacement d of this body after time t is d = vt + 1/2at^2
        displacement = self.velocity * self.time + (1/2) * bird_acceleration * self.time ** 2 #calculate the displacement when going downward
        
        #if the bird is going down too fast, we just take the maximum displacement
        if displacement > bird_max_displacement:
            displacement = bird_max_displacement
        
        self.y = self.y + displacement #calculate the new bird y position after the displacement
        
        if displacement < 0: #or self.y < self.fly_height + 50: #when the bird is going up
            if self.fly_degree < self.MIN_UPWARD_ROTATION_DEGREE: #if the fly degree is less than the minimum fly upward degree
                self.fly_degree = self.MIN_UPWARD_ROTATION_DEGREE #set the fly degree to be MIN_UPWARD_ROTATION_DEGREE
            elif self.fly_degree > 90:
                self.fly_degree = 90
                
        else: #when the bird is going down
            if self.fly_degree > -90: #if the bird's fly degree is too small
                self.fly_degree -= self.DOWNWARD_ROTATION_VELOCITY #accelerate it to approach 90 degrees downward by DOWNWARD_ROTATION_VELOCITY
            elif self.fly_degree <= -90:
                self.fly_degree = -90

    #defien a function to jump the bird
    def jump(self):
        self.velocity = bird_jump_velocity #jump up by bird_jump_velocity
        self.time = 0 #when we jump, we reset the time to 0
        self.fly_height = self.y #the current height of the bird before jump is the y position
    
    #define a function to draw the bird
    def draw_surface(self, window):
        self.animation_time_count += 1
        #if the bird is diving, then it shouldn't flap its wings        
        if self.fly_degree < 0:
            self.bird_img = self.IMGS[0]
            self.animation_time_count = 0 #reset the animation_time_count
        
        #if the bird is not diving, then it should flap its wings
        #keep looping the 3 bird images to mimic flapping its wings
        elif self.animation_time_count < self.FLAP_TIME:
            self.bird_img = self.IMGS[0]
        elif self.animation_time_count < self.FLAP_TIME * 2:
            self.bird_img = self.IMGS[1]
        elif self.animation_time_count < self.FLAP_TIME * 3:
            self.bird_img = self.IMGS[2]
        elif self.animation_time_count < self.FLAP_TIME * 4:
            self.bird_img = self.IMGS[1]
        else:
            self.bird_img = self.IMGS[0]
            self.animation_time_count = 0 #reset the animation_time_count
        
        #https://stackoverflow.com/questions/4183208/how-do-i-rotate-an-image-around-its-center-using-pygame
        #start to draw the bird
        #rotate the bird image for degree at self.tilt
        rotated_image = pygame.transform.rotate(self.bird_img, self.fly_degree)
        #store the center of the source image rectangle
        origin_img_center = self.bird_img.get_rect(topleft = (self.x, self.y)).center
        #update the center of the rotated image rectangle
        rotated_rect = rotated_image.get_rect(center = origin_img_center)
        #draw the rotated bird image on rotated rectangle
        window.blit(rotated_image, rotated_rect)

#define a function to check collision
def collide(bird, pipe, floor, window):
    
    #Creates a Mask object from the given surface by setting all the opaque pixels and not setting the transparent pixels
    bird_mask = pygame.mask.from_surface(bird.bird_img) #get the mask of the bird
    top_pipe_mask = pygame.mask.from_surface(pipe.top_pipe) #get the mask of the pipe on the top
    bottom_pipe_mask = pygame.mask.from_surface(pipe.bottom_pipe) #get the mask of the pipe on the bottom
    
    sky_height = 0 #the sky height is the upper limit
    floor_height = floor.y #the floor height is the lower limit
    bird_lower_end = bird.y + bird.bird_img.get_height() #the y position of the lower end of the bird image
    
    #in order to check whether the bird hit the pipe, we need to find the point of intersection of the bird and the pipes
    #if overlap, then mask.overlap(othermask, offset) return (x, y)
    #if not overlap, then mask.overlap(othermask, offset) return None
    #more information regarding offset, https://www.pygame.org/docs/ref/mask.html#mask-offset-label
    top_pipe_offset = (round(pipe.x - bird.x), round(pipe.top_pipe_topleft - bird.y))
    bottom_pipe_offset = (round(pipe.x - bird.x), round(pipe.bottom_pipe_topleft - bird.y))
    
    #Returns the first point of intersection encountered between bird's mask and pipe's masks
    top_pipe_intersection_point = bird_mask.overlap(top_pipe_mask, top_pipe_offset)
    bottom_pipe_intersection_point = bird_mask.overlap(bottom_pipe_mask, bottom_pipe_offset)

    if  top_pipe_intersection_point is not None or bottom_pipe_intersection_point is not None or bird_lower_end > floor_height or bird.y < sky_height:
        return True
    else:
        return False

#define a function to get the input index of the pipes
def get_index(pipes, birds):
    #get the birds' x position
    bird_x = birds[0].x
    #calculate the x distance between birds and each pipes
    list_distance = [pipe.x + pipe.IMG_WIDTH - bird_x for pipe in pipes]
    #get the index of the pipe that has the minimum non negative distance(the closest pipe in front of the bird)
    index = list_distance.index(min(i for i in list_distance if i >= 0)) 
    return index

#define a function to draw the window to display the game
def draw_game(window, birds, pipes, floor, score, generation, game_time):
    
    window.blit(BG_IMG, (0, 0)) #draw the background
    
    for bird in birds:
        bird.draw_surface(window) #draw the birds

    for pipe in pipes:
        pipe.draw_surface(window) #draw the pipes
    
    floor.draw_surface(window) #draw the floor
    
    score_text = FONT.render('Score: ' + str(score), 1, (255, 255, 255)) #set up the text to show the scores
    window.blit(score_text, (WINDOW_WIDTH - 15 - score_text.get_width(), 15)) #draw the scores
    
    game_time_text = FONT.render('Timer: ' + str(game_time) + ' s', 1, (255, 255, 255)) #set up the text to show the progress
    window.blit(game_time_text, (WINDOW_WIDTH - 15 - game_time_text.get_width(), 15 + score_text.get_height())) #draw the progress
    
    generation_text = FONT.render('Generation: ' + str(generation - 1), 1, (255, 255, 255)) #set up the text to show the number of generation
    window.blit(generation_text, (15, 15)) #draw the generation
    
    bird_text = FONT.render('Birds Alive: ' + str(len(birds)), 1, (255, 255, 255)) #set up the text to show the number of birds alive
    window.blit(bird_text, (15, 15 + generation_text.get_height())) #draw the number of birds alive
    
    progress_text = FONT.render('Pipes Remained: ' + str(len(pipes) - score), 1, (255, 255, 255)) #set up the text to show the progress
    window.blit(progress_text, (15, 15 + generation_text.get_height() + bird_text.get_height())) #draw the progress
    
    pygame.display.update() #show the surface

#import packages to build the AI
import neat

#define a function to run NEAT algorithm to play flappy bird
def run_AI(config_file):
    
    #the template for the configuration file can be found here:
    #https://github.com/CodeReclaimers/neat-python/blob/master/examples/xor/config-feedforward
    #the description of the options in the configuration file can be found here:
    #https://neat-python.readthedocs.io/en/latest/config_file.html#defaultgenome-section
    
    #use NEAT algorithm to build a neural network based on the pre-set configurtion
    #Create a neat.config.Config object from the configuration file
    config = neat.config.Config(neat.DefaultGenome, 
                                neat.DefaultReproduction,
                                neat.DefaultSpeciesSet, 
                                neat.DefaultStagnation,
                                config_file)
    
    #Create a neat.population.Population object using the Config object created above
    neat_pop = neat.population.Population(config)
    
    #show the summary statistics of the learning progress
    neat_pop.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    neat_pop.add_reporter(stats)
    
    #Call the run method on the Population object, giving it your fitness function and (optionally) the maximum number of generations you want NEAT to run
    neat_pop.run(main, MAX_GEN)
    
    #get the most fit genome genome as our winner with the statistics.best_genome() function
    winner = stats.best_genome()
    
    #show the final statistics
    print('\nBest genome:\n{!s}'.format(winner))

#define a function to run the main game loop
def main(genomes, config):
    
    global generation, WINDOW #use the global variable gen and WINDOW
    window = WINDOW
    generation += 1 #update the generation
    
    score = 0 #initiate score to 0
    clock = pygame.time.Clock() #set up a clock object to help control the game framerate
    start_time = pygame.time.get_ticks() #reset the start_time after every time we update our generation
    
    floor = Floor(floor_starting_y_position) #build the floor
    pipes_list = [Pipe(pipe_starting_x_position + i * pipe_horizontal_gap) for i in range(pipe_max_num)] #build the pipes and seperate them by pipe_horizontal_gap
    
    models_list = [] #create an empty list to hold all the training neural networks
    genomes_list = [] #create an empty list to hold all the training genomes
    birds_list = [] #create an empty list to hold all the training birds
    
    for genome_id, genome in genomes: #for each genome
        birds_list.append(Bird(bird_starting_x_position, bird_starting_y_position)) #create a bird and append the bird in the list
        genome.fitness = 0 #start with fitness of 0
        genomes_list.append(genome) #append the genome in the list
        model = neat.nn.FeedForwardNetwork.create(genome, config) #set up the neural network for each genome using the configuration we set
        models_list.append(model) #append the neural network in the list
        
    run = True
    
    while run is True: #when we run the program
        
        #check the event of the game and quit if we close the window
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
                break
        
        #stop the game when the score exceed the maximum score
        #break the loop and restart when no bird left
        if score >= max_score or len(birds_list) == 0:
            run = False
            break
        
        game_time = round((pygame.time.get_ticks() - start_time)/1000, 2) #record the game time for this generation
        
        clock.tick(FPS) #update the clock, run at FPS frames per second (FPS). This can be used to help limit the runtime speed of a game.
        
        floor.move() #move the floor
        
        pipe_input_index = get_index(pipes_list, birds_list) #get the input index of the pipes list
        
        passed_pipes = [] #create an empty list to hold all the passed pipes
        for pipe in pipes_list:
            pipe.move() #move the pipe
            if pipe.x + pipe.IMG_WIDTH < birds_list[0].x: #if the bird passed the pipe
                passed_pipes.append(pipe) #append the pipe to the passed pipes list
                for genome in genomes_list:
                    genome.fitness += pass_reward #increase the fitness of each remained genome by pass_reward to reward them for successfully passed the pipe 
        
        score = len(passed_pipes) #calculate the score of the game, which equals to the number of pipes the bird passed
        
        for x, bird in enumerate(birds_list):
            genomes_list[x].fitness += stay_reward #increase the fitness of each remained genome to reward for staying longer
            bird.move() #move the bird
            delta_x = bird.x - pipes_list[pipe_input_index].x #input 1: the horizontal distance between the bird and the pipe
            delta_y_top = bird.y - pipes_list[pipe_input_index].top_pipe_height #input 2: the vertical distance between the bird and the top pipe
            delta_y_bottom = bird.y - pipes_list[pipe_input_index].bottom_pipe_topleft #input 3: the vertical distance between the bird and the bottom pipe
            net_input = (delta_x, delta_y_top, delta_y_bottom)
            #input the bird's distance from the pipes to get the output of whether to jump or not
            output = models_list[x].activate(net_input)
            
            if output[0] > prob_threshold_to_jump: #if the model output is greater than the probability threshold to jump
                bird.jump() #then jump the bird

        for index, bird in enumerate(birds_list):
            if collide(bird, pipes_list[pipe_input_index], floor, window) is True: #check collision for each bird in the population
                genomes_list[index].fitness -= failed_punishment #reduce the fitness by failed_punishment if collided
                models_list.pop(index) #drop the model from the list if collided
                genomes_list.pop(index) #drop the genome from the list if collided
                birds_list.pop(index) #drop the bird from the list if collided

        draw_game(window, birds_list, pipes_list, floor, score, generation, game_time) #draw the window of the game

#run the game!
config_file = 'config-feedforward.txt'
run_AI(config_file)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html

 ****** Running generation 0 ****** 



error: display Surface quit

In [1]:
#import packages to build the game
import pygame
import time
import os
import random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#initialize pygame
pygame.init()

#set up the window and floor
WIN_WIDTH = 550
WIN_HEIGHT = 550
FLOOR_HEIGHT = 500
WIN = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))

#load the images
BIRD_IMGS = [pygame.image.load('bird1.png'),
             pygame.image.load('bird2.png'),
             pygame.image.load('bird3.png')]
BOTTOM_PIPE_IMG = pygame.image.load('pipe.png')
TOP_PIPE_IMG = pygame.transform.flip(BOTTOM_PIPE_IMG, False, True) #flip the image of the bottom pipe to get the image for the pipe on the top
BASE_IMG = pygame.image.load('base.png')
BG_IMG = pygame.image.load('bg.png')

#set the font
FONT = pygame.font.SysFont('comicsansms', 25)

#set the start of the generation of AI
gen = 0

#set the game options
FPS = 30 #run the game at rate FPS, the speed at which images are shown

#base options
base_velocity = 5 #the horizontal moving velocity of the base, this should equal to pipe_velocity

#pipe options
pipe_gap = 150 #the gap between the top pipe and the bottom pipe, the smaller the number, the harder the game
pipes_gap = WIN_WIDTH//2 + 100 #the gap between two sets of pipes
pipe_velocity = 5 #the horizontal moving velocity of the pipes, this should equal to base_velocity
top_pipe_min_height = 100 #the minimum height of the top pipe (carefully set this number)
top_pipe_max_height = 300 #the maximum height of the top pipe (carefully set this number)
pipe_starting_x_position = 500 #the starting x position of the pipe

#bird options
bird_max_rotation = 25 #the maximum rotation degree
bird_rotation_velocity = 20 #the maximum rotation velocity
bird_animation_time = 2 #the animation time of showing one image
bird_jump_velocity = -8 #the vertical jump up velocity
bird_acceleration = 3 #the gravity for the bird in the game
bird_max_displacement = 12 #the maximum displacement per frame
bird_starting_x_position = 150 #the starting x position of the bird
bird_starting_y_position = 250 #the starting y position of the bird

In [3]:
#build the class Base
class Base:
    #Base's attributes
    VEL = base_velocity #the moving velocity of the base
    WIDTH = BASE_IMG.get_width() #the width of the base
    IMG = BASE_IMG #the image of the base

    #initialize the Object
    def __init__(self, y):
        #since our base is not wide enough to fill the screen, we need 3 bases to set up the moving base
        #these 3 bases have different starting position but have the same y position
        self.base1 = BASE_IMG
        self.base2 = BASE_IMG
        self.base3 = BASE_IMG
        self.y = y #the y position of the base
        self.x1 = 0 #the starting x position of the first base
        self.x2 = self.WIDTH #the starting x position of the second base
        self.x3 = self.WIDTH * 2 #the starting x position of the third base
        
    #define a function to move the base
    def move(self):
        self.x1 -= self.VEL #move to the left with the velocity of VEL
        self.x2 -= self.VEL #move to the left with the velocity of VEL
        self.x3 -= self.VEL #move to the left with the velocity of VEL
        
        if self.x1 + self.WIDTH < 0: #if the first base moves out of the window 
            self.x1 = self.x3 + self.WIDTH #then move the first base to to the right of the third base
        if self.x2 + self.WIDTH < 0: #if the second base moves out of the window 
            self.x2 = self.x1 + self.WIDTH #then move the second base to to the right of the first base
        if self.x3 + self.WIDTH < 0: #if the third base moves out of the window 
            self.x3 = self.x2 + self.WIDTH #then move the third base to to the right of the second base
            
    #define a function to draw the base
    def draw(self, win):
        win.blit(self.base1, (self.x1, self.y)) #draw the first base
        win.blit(self.base2, (self.x2, self.y)) #draw the second base
        win.blit(self.base3, (self.x3, self.y)) #draw the third base

In [4]:
#build the class Pipe
class Pipe:
    #Pipe's attributes
    GAP = pipe_gap #the gap between the top and bottom pipes
    VEL = pipe_velocity #the moving velocity of the pipes
    WIDTH = TOP_PIPE_IMG.get_width() #the width of the pipe

    #initialize the Object
    def __init__(self, x):                
        self.x = x #starting x position of the pipes
        self.height = 0 #the starting height of the top pipe
        self.top = 0 #the starting topleft position of the top pipe
        self.bottom = 0 #the starting topleft position of the bottom pipe
        self.PIPE_TOP = TOP_PIPE_IMG #get the image for the pipe on the top
        self.PIPE_BOTTOM = BOTTOM_PIPE_IMG #get the image for the pipe on the bottom
        self.passed = False #check whether the bird has passed the pipe
        self.set_height() #set the height of the pipes randomly as well as the starting topleft position for top and bottom pipes
        
    #define a function to move the pipe
    def move(self):
        self.x -= self.VEL
    
    def set_height(self):
        self.height = random.randrange(top_pipe_min_height, top_pipe_max_height) #the range is between top_pipe_min_height and top_pipe_max_height
        self.top = self.height - self.PIPE_TOP.get_height() #the topleft position of the top pipe should be the random height - the length of the pipe
        self.bottom = self.height + self.GAP #the topleft position of the bottom pipe should be the random height + the GAP

    #define a function to draw the pipe
    def draw(self, win):
        win.blit(self.PIPE_TOP, (self.x, self.top)) #draw the pipe on the top
        win.blit(self.PIPE_BOTTOM, (self.x, self.bottom)) #draw the pipe on the bottom

In [5]:
#build the class Bird
class Bird:
    #Bird's attributes
    IMGS = BIRD_IMGS
    MAX_ROTATION = bird_max_rotation
    ROT_VEL = bird_rotation_velocity
    ANIMATION_TIME = bird_animation_time
    
    #initialize the Object
    def __init__(self, x_position, y_position):
        self.x = x_position #starting x position
        self.y = y_position #starting y position
        self.tilt = 0 #starting degree
        self.tick_count = 0 #starting time set to calculate displacement
        self.vel = 0 #starting velocity
        self.height = self.y #starting height of the bird
        self.img_count = 0 #used to change bird images
        self.img = self.IMGS[0] #use the first image as the initial image
    
    #define a function to draw the bird
    def draw(self, win):
        self.img_count += 1
        #if the bird is diving, then it shouldn't flap its wings        
        if self.tilt < 0:
            self.img = self.IMGS[1]
            self.img_count = 0 #reset the img_count
        
        #if the bird is not diving, then it should flap its wings
        #keep looping the 3 bird images to mimic flapping its wings
        elif self.img_count < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count < self.ANIMATION_TIME * 2:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION_TIME * 3:
            self.img = self.IMGS[2]
        elif self.img_count < self.ANIMATION_TIME * 4:
            self.img = self.IMGS[1]
        else:
            self.img = self.IMGS[0]
            self.img_count = 0 #reset the img_count
        
        #https://stackoverflow.com/questions/4183208/how-do-i-rotate-an-image-around-its-center-using-pygame
        #start to draw the bird
        #rotate the bird image for degree at self.tilt
        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        #store the center of the source image rectangle
        origin_img_center = self.img.get_rect(topleft = (self.x, self.y)).center
        #update the center of the rotated image rectangle
        rotated_rect = rotated_image.get_rect(center = origin_img_center)
        #draw the rotated bird image on rotated rectangle
        win.blit(rotated_image, rotated_rect)

    #defien a function to jump the bird
    def jump(self):
        self.vel = bird_jump_velocity #jump up by bird_jump_velocity
        self.tick_count = 0 #when we jump, we reset the tick count to 0
        self.height = self.y #the current height of the bird before jump is the y position
        
    #defien a function to move the bird
    def move(self):
        self.tick_count += 1 #count the time
        
        #for a body with a nonzero speed v and a constant acceleration a
        #the displacement d of this body after time t is d = vt + 1/2at^2
        displacement = self.vel * self.tick_count + (1/2) * bird_acceleration * self.tick_count ** 2 #calculate the displacement when going downward
        
        #if the bird is going down too fast, we just take the maximum displacement
        if displacement > bird_max_displacement:
            displacement = bird_max_displacement
        
        #if displacement < 0:
        #    displacement -= 2
        
        self.y = self.y + displacement #calculate the new bird y position after the displacement
        
        if displacement < 0 or self.y < self.height + 50: #when the bird is going up
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION
            #self.tilt = self.MAX_ROTATION if self.tilt < self.MAX_ROTATION #the degree to tilt up equals to the maximum tilt up degrees
        else: #when the bird is going down
            if self.tilt > -90:
                self.tilt -= self.ROT_VEL
            #self.tilt -= self.ROT_VEL if self.tilt > -90 #the degree to tilt down equals to 90

In [6]:
#define a function to check collision
def collide(bird, pipe, base, win):
    bird_mask = pygame.mask.from_surface(bird.img) #get the mask of the bird
    top_mask = pygame.mask.from_surface(pipe.PIPE_TOP) #get the mask of the pipe on the top
    bottom_mask = pygame.mask.from_surface(pipe.PIPE_BOTTOM) #get the mask of the pipe on the bottom
        
    sky_height = 0
    floor_height = base.y
    bird_lower_end = bird.y + bird.img.get_height()
        
    top_offset = (pipe.x - bird.x, pipe.top - round(bird.y))
    bottom_offset = (pipe.x - bird.x, pipe.bottom - round(bird.y))

    b_point = bird_mask.overlap(bottom_mask, bottom_offset)
    t_point = bird_mask.overlap(top_mask, top_offset)

    if b_point or t_point or bird_lower_end > floor_height or bird.y < sky_height:
        return True
    else:
        return False

In [7]:
def get_index(pipes, birds):
    bird_x = birds[0].x
    list_distance = [pipe.x + pipe.PIPE_TOP.get_width() - bird_x for pipe in pipes]
    index = list_distance.index(min(i for i in list_distance if i >= 0)) 
    return index

In [8]:
#define a function to draw the window
def draw_window(win, birds, pipes, base, score, gen):
    
    if gen == 0:
        gen = 1
    #gen = 1 if gen == 0 #set the generation start from 1
    
    win.blit(BG_IMG, (0, 0)) #draw the background
    
    base.draw(win) #draw the base
    
    for pipe in pipes:
        pipe.draw(win)
    
    #for pipe in pipes:
     #   pipe.draw(win) #draw the pipes

    for bird in birds:
        bird.draw(win) #draw the birds
    
    score_text = FONT.render('Score: ' + str(score), 1, (255, 255, 255)) #set up the text to show the scores
    win.blit(score_text, (WIN_WIDTH - 10 - score_text.get_width(), 10)) #draw the scores
    
    generation_text = FONT.render('Generation: ' + str(gen), 1, (255, 255, 255)) #set up the text to show the number of generation
    win.blit(generation_text, (10, 10)) #draw the generation
    
    bird_text = FONT.render('Alive: ' + str(len(birds)), 1, (255, 255, 255)) #set up the text to show the number of birds alive
    win.blit(bird_text, (10, 10 + generation_text.get_height())) #draw the number of alive
    
    pygame.display.update() #show the surface

In [9]:
#import packages to build the AI
import pickle
import neat

In [10]:
#define a function the run the game
def run(config_file):
    #use NEAT algorithm to build a neural network based on the pre-set configurtion
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                                neat.DefaultSpeciesSet, neat.DefaultStagnation,
                                config_file)
    
    #create the population for the NEAT model
    p = neat.Population(config)
    
    #show the summary statistics of the learning progress
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    #run for up to 50 generations
    winner = p.run(main, 50)
    
    #show the final statistics
    print('\nBest genome:\n{!s}'.format(winner))

In [11]:
#define a function to run the AI simulation
def main(genomes, config):
    global gen, WIN #use the global variable gen and WIN
    win = WIN
    gen += 1 #update the generation
    
    nets = [] #create an empty list to hold all the training neural networks
    ge = [] #create an empty list to hold all the training genomes
    birds = [] #create an empty list to hold all the training birds
    
    for genome_id, genome in genomes: #for each genome
        birds.append(Bird(bird_starting_x_position, bird_starting_y_position)) #create a bird and append the bird in the list
        genome.fitness = 0 #start with fitness of 0
        ge.append(genome) #append the genome in the list
        net = neat.nn.FeedForwardNetwork.create(genome, config) #set up the neural network for each genome using the configuration we set
        nets.append(net) #append the neural network in the list
    
    base = Base(FLOOR_HEIGHT) #build the base
    
    pipes = []

    for i in range(100):
        pipe = Pipe(pipe_starting_x_position + i * pipes_gap)
        pipes.append(pipe)
        
    clock = pygame.time.Clock() #set up the clock to record our game time
    score = 0 #initiate score to 0
    
    run = True
    
    while run is True and len(birds) > 0: #when we run the program and we have at least one bird
        
        #check the event of the game and quit if needed
        for event in pygame.event.get():
            if event.type == pygame.QUIT or len(birds) == 0:
                run = False
                pygame.quit()
                quit()
                break
        clock.tick(FPS) #run at 30 frames per second (FPS)
        base.move() #move the base
        
        #stop the game when the score exceed 50
        if score > 50:
            #save the best AI
            pickle.dump(nets[0], open('AI Model for Flappy Bird.pickle', 'wb'))
            break
        
        pipe_input_index = get_index(pipes, birds)
        
        #num_pipes = len(pipes) #check the number of pipes on the screen
        #if num_pipes == 1: #if we only have one set of pipe
        #    pipe_input_index = 0 #input the first pipe location to our model
        #elif num_pipes > 1 and birds[0].x > pipes[0].PIPE_TOP.get_width(): #if we have more than 1 sets of pipes on the screen and the birds passed the first set of pipes
        #    pipe_input_index = 1 #input the second pipe location to our model
        
        passed_pipes = []
        for pipe in pipes:
            pipe.move()
            if pipe.x < birds[0].x:
                passed_pipes.append(pipe)
                for genome in ge:
                    genome.fitness += 5
        
        score = len(passed_pipes)
        
        for x, bird in enumerate(birds):
            ge[x].fitness += 0.1 #increase the fitness of the genome to reward for staying longer
            bird.move() #move the bird
            #if pipe.x1 - bird.x >= 0 and pipe.x1 - bird.x < pipe.x2 - bird.x:
            #    delta_x = bird.x - pipe.x1
            #    delta_y_top = bird.y - pipe.height1
            #    delta_y_bottom = bird.y - pipe.bottom1
            #else:
            #    delta_x = bird.x - pipe.x2
            #    delta_y_top = bird.y - pipe.height2
            #    delta_y_bottom = bird.y - pipe.bottom2
            delta_x = bird.x - pipes[pipe_input_index].x
            delta_y_top = bird.y - pipes[pipe_input_index].height
            delta_y_bottom = bird.y - pipes[pipe_input_index].bottom
            net_input = (delta_x, delta_y_top, delta_y_bottom)
            #input the bird's y location and the distance from the pipes to get the output of whether to jump or not
            output = nets[x].activate(net_input)
            
            if output[0] > 0.5: #if the model output is greater than the 0.5 threshold
                bird.jump() #then jump

        for index, bird in enumerate(birds):
            if collide(bird, pipes[pipe_input_index], base, win) is True:
                ge[index].fitness -= 1
                nets.pop(index)
                ge.pop(index)
                birds.pop(index)

        draw_window(win, birds, pipes, base, score, gen) #draw the window of the game

In [12]:
#run the game!
config_file = 'config-feedforward.txt'
run(config_file)


 ****** Running generation 0 ****** 

Population's average fitness: 3.83333 stdev: 1.17851
Best fitness: 5.50000 - size: (1, 3) - species 1 - id 2
Average adjusted fitness: 0.333
Mean genetic distance 1.537, standard deviation 0.782
Population of 3 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     3      5.5    0.333     0
Total extinctions: 0
Generation time: 2.216 sec

 ****** Running generation 1 ****** 



error: display Surface quit